In [12]:
# 1 Importing Dependencies
 
import pandas as pd
import requests
import json
import sys
from pymongo import MongoClient
sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import nba_rapidapi_key
import time

In [2]:
# 2 API Calling in Seasons

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data"]
collection = db["seasons"]

if "seasons" in db.list_collection_names():
    db.drop_collection("seasons")
    print("Existing 'seasons' collection dropped.")

url = "https://api-nba-v1.p.rapidapi.com/seasons"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
response_data = response.json()

print(response_data)

for season in response_data['response']:
    season_document = {"season": season}
    collection.insert_one(season_document)
    print(f"Inserted season {season} into MongoDB")


Existing 'seasons' collection dropped.
{'get': 'seasons/', 'parameters': [], 'errors': [], 'results': 9, 'response': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]}
Inserted season 2015 into MongoDB
Inserted season 2016 into MongoDB
Inserted season 2017 into MongoDB
Inserted season 2018 into MongoDB
Inserted season 2019 into MongoDB
Inserted season 2020 into MongoDB
Inserted season 2021 into MongoDB
Inserted season 2022 into MongoDB
Inserted season 2023 into MongoDB


In [3]:
# 3 API Calling in Teams

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data"]
collection = db["teams"]

if "teams" in db.list_collection_names():
    db.drop_collection("teams")
    print("Existing 'teams' collection dropped.")

url = "https://api-nba-v1.p.rapidapi.com/teams"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
response_data = response.json()

print(response_data)

for team in response_data['response']:
    if team['nbaFranchise'] and team['allStar']==False:
        team_document = {"team": team}
        collection.insert_one(team_document)
        print(f"Inserted season {team} into MongoDB")

Existing 'teams' collection dropped.
{'get': 'teams/', 'parameters': [], 'errors': [], 'results': 66, 'response': [{'id': 1, 'name': 'Atlanta Hawks', 'nickname': 'Hawks', 'code': 'ATL', 'city': 'Atlanta', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/e/ee/Hawks_2016.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Southeast'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Southeast'}, 'sacramento': {'conference': 'East', 'division': 'Southeast'}}}, {'id': 2, 'name': 'Boston Celtics', 'nickname': 'Celtics', 'code': 'BOS', 'city': 'Boston', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/thumb/6/65/Celtics_de_Boston_logo.svg/1024px-Celtics_de_Boston_logo.svg.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Atlantic'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Atla

In [4]:
# 4 API Calling in 2021-2023 Games

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data"]
collection = db["games"]

if "games" in db.list_collection_names():
    db.drop_collection("games")
    print("Existing 'games' collection dropped.")

url = "https://api-nba-v1.p.rapidapi.com/games"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

season_list = ["2021", "2022", "2023"]

for season in season_list:
    querystring = {"season": season}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        response_data = response.json()
        for game in response_data['response']:
            game_document = {"season": season, "game": game}
            collection.insert_one(game_document)
            print(f"Inserted game from season {season} into MongoDB")
    else:
        print(f"Failed to fetch data for season {season}, status code: {response.status_code}")

Existing 'games' collection dropped.
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into Mon

In [3]:
# Code to get list of game IDs from all games 

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data"]
collection = db["games"]

query = {}
projection = {'game.id':1, '_id': 0}

cursor = collection.find(query, projection)

game_ids= [doc['game']['id'] for doc in cursor]
df = pd.DataFrame(game_ids, columns=['game_ids'])

game_id_list = df['game_ids'].tolist()

mongo.close()

print(game_id_list)

[8787, 8788, 8789, 8790, 8791, 8792, 8793, 8846, 8847, 8848, 8849, 8850, 8851, 8893, 8894, 8895, 8896, 8897, 8898, 8899, 8900, 8901, 8902, 8903, 8904, 8905, 8906, 8907, 8908, 8909, 8910, 8911, 8912, 8913, 8914, 8915, 8916, 8917, 8918, 8919, 8920, 8921, 8922, 8923, 8924, 8925, 8926, 8927, 8928, 8929, 8930, 8931, 8932, 8933, 8934, 8935, 8936, 8937, 8938, 8939, 9094, 9095, 9096, 9097, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9105, 9106, 9107, 9108, 9109, 9110, 9111, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9120, 9121, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140, 9141, 9142, 9143, 9144, 9145, 9146, 9147, 9148, 9149, 9150, 9151, 9152, 9153, 9154, 9155, 9156, 9157, 9158, 9159, 9160, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9173, 9174, 9175, 9176, 9177, 9178, 9179, 9180, 9181, 9182, 9183, 9184, 9185, 9186, 9187, 9188, 9189, 9190, 9191, 9192, 9193, 9194, 9195, 9196, 9197, 9198, 9199, 9200, 9201, 920

In [10]:
len(game_id_list)

4346

In [11]:
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data"]
collection = db["game_stats"]

if "game_stats" in db.list_collection_names():
    db.drop_collection("game_stats")
    print("Existing 'game_stats' collection dropped.")

url = "https://api-nba-v1.p.rapidapi.com/games/statistics"

headers = {
	"X-RapidAPI-Key": "bb6c1e3fe9mshf016f82ca36e8c6p19fd11jsn48c79b55dc12",
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

short_list = [8787, 8788, 8789, 8790, 8791]

for game_id in short_list:
    querystring = {"id":game_id}
    response = requests.get(url, headers=headers, params=querystring)
    response_data = response.json()
    collection.insert_one(response_data)




Existing 'game_stats' collection dropped.
